In [1]:
!pip3 install --upgrade --quiet langchain langchain-community langchain-openai chromadb 
!pip3 install --upgrade --quiet pypdf pandas streamlit python-dotenv

zsh:1: /Users/apple/Documents/rag_llms/myenv/bin/pip3: bad interpreter: /Users/apple/Documents/rag_llms/venv/bin/python3.13: no such file or directory

[notice] A new release of pip available: 22.3 -> 26.0.1
[notice] To update, run: /usr/local/bin/python3 -m pip install --upgrade pip
zsh:1: /Users/apple/Documents/rag_llms/myenv/bin/pip3: bad interpreter: /Users/apple/Documents/rag_llms/venv/bin/python3.13: no such file or directory
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3 -> 26.0.1
[notice] To update, run: /usr/local/bin/python3 -m pip install --upgrade pip


In [2]:
# LangChain - Fully updated imports (2025-safe)
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

# Pydantic (direct)
from pydantic import BaseModel, Field

# Other modules
import os
import tempfile
import streamlit as st
import pandas as pd
from dotenv import load_dotenv


In [3]:
load_dotenv()

False

In [26]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

Define our LLM

In [27]:
llm = ChatOpenAI(model="gpt-4.1-mini", api_key=OPENAI_API_KEY)
llm.invoke("Tell me a joke about cats")

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

Process PDF document


Load PDF document

In [8]:
loader = PyPDFLoader("data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf")
pages = loader.load()
pages

Ignoring wrong pointing object 18 0 (offset 0)


[Document(metadata={'producer': 'macOS Version 14.4.1 (Build 23E224) Quartz PDFContext, AppendMode 1.1', 'creator': 'Preview', 'creationdate': "D:20240909152042Z00'00'", 'author': 'Thu Vu', 'moddate': "D:20240910141854Z00'00'", 'title': 'Oppenheimer-2006-Applied_Cognitive_Psychology', 'source': 'data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='APPLIED COGNITIVE PSYCHOLOGY\nAppl. Cognit. Psychol. 20: 139–156 (2006)\nPublished online 31 October 2005 in Wiley InterScience\n(www.interscience.wiley.com) DOI: 10.1002/acp.1178\nConsequences of Erudite Vernacular Utilized Irrespective\nof Necessity: Problems with Using Long Words Needlessly\nDANIEL M. OPPENHEIMER*\nPrinceton University, USA\nSUMMARY\nMost texts on writing style encourage authors to avoid overly-complex words. However, a majority\nof undergraduates admit to deliberately increasing the complexity of their vocabulary so as to give\nthe impression of intelligen

Split document

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                            chunk_overlap=200,
                                            length_function=len,
                                            separators=["\n\n", "\n", " "])
chunks = text_splitter.split_documents(pages)

Create embeddings

In [10]:
def get_embedding_function():
    embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002", openai_api_key=OPENAI_API_KEY
    )
    return embeddings

embedding_function = get_embedding_function()
test_vector = embedding_function.embed_query("cat")

In [11]:
from langchain_openai import OpenAIEmbeddings
import numpy as np

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vec1 = embeddings.embed_query("Amsterdam")
vec2 = embeddings.embed_query("coffeeshop")


In [12]:
def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

score = cosine_similarity(vec1, vec2)
print("Similarity score:", score)


Similarity score: 0.29002255800906246


Create vector database

In [13]:
import uuid

def create_vectorstore(chunks, embedding_function, vectorstore_path):

    # Create a list of unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]
    
    # Ensure that only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []
    
    unique_chunks = [] 
    for chunk, id in zip(chunks, ids):     
        if id not in unique_ids:       
            unique_ids.add(id)
            unique_chunks.append(chunk) 

    # Create a new Chroma database from the documents
    vectorstore = Chroma.from_documents(documents=unique_chunks, 
                                        ids=list(unique_ids),
                                        embedding=embedding_function, 
                                        persist_directory = vectorstore_path)

    vectorstore.persist()
    
    return vectorstore

Query for relevant data

In [14]:
# Create vectorstore
vectorstore = create_vectorstore(chunks=chunks, 
                                 embedding_function=embedding_function, 
                                 vectorstore_path="vectorstore_test")

/var/folders/3j/6dxlknmd5hs82yqpxtj2n3z00000gn/T/ipykernel_13160/2260934871.py:24: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [30]:
vectorstore = Chroma(
    persist_directory="vectorstore_test",
    embedding_function=embedding_function
)


In [31]:
# Create retriever and get relevant chunks
retriever = vectorstore.as_retriever(search_type="similarity")
relevant_chunks = retriever.invoke("What is the title of the paper?")
relevant_chunks

[Document(metadata={'page': 1, 'creationdate': "D:20240909152042Z00'00'", 'producer': 'macOS Version 14.4.1 (Build 23E224) Quartz PDFContext, AppendMode 1.1', 'title': 'Oppenheimer-2006-Applied_Cognitive_Psychology', 'moddate': "D:20240910141854Z00'00'", 'total_pages': 3, 'source': 'data/Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'page_label': '2', 'creator': 'Preview', 'author': 'Thu Vu'}, page_content='was unnecessary and thus surprising readers with the relative disﬂuency of the text.\nBoth the experts and prevailing wisdom present plausible views, but which (if either) is\ncorrect? The present paper provides an empirical investigation of the strategy of complex-\nity, and ﬁnds such a strategy to be unsuccessful. Five studies demonstrate that the loss of\nﬂuency due to needless complexity in a text negatively impacts raters’ assessments of the\ntext’s authors.\nEXPERIMENT 1\nExperiment 1 aimed to answer several simple questions. First, does increasing the\ncomplexity of tex

In [32]:
# Prompt template
PROMPT_TEMPLATE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

{context}

---

Answer the question based on the above context: {question}
"""

Generate responses

In [33]:
	
# Concatenate context text
context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

# Create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, 
                                question="What is the title of the paper?")
print(prompt)

Human: 
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

was unnecessary and thus surprising readers with the relative disﬂuency of the text.
Both the experts and prevailing wisdom present plausible views, but which (if either) is
correct? The present paper provides an empirical investigation of the strategy of complex-
ity, and ﬁnds such a strategy to be unsuccessful. Five studies demonstrate that the loss of
ﬂuency due to needless complexity in a text negatively impacts raters’ assessments of the
text’s authors.
EXPERIMENT 1
Experiment 1 aimed to answer several simple questions. First, does increasing the
complexity of text succeed in making the author appear more intelligent? Second, to
what extent does the success of this strategy depend on the quality of the original, simpler
writing? Finally, if the strategy is unsuccessful, is th

In [34]:
llm.invoke(prompt)

AIMessage(content='The title of the paper is "Consequences of Erudite Vernacular Utilized Irrespective of Necessity: Problems with Using Long Words Needlessly."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 1114, 'total_tokens': 1146, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_82dbabdb2c', 'id': 'chatcmpl-D5vYcLIyjaVtYc8cQRh0D8FyybOhW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2e6a-8ded-7dc2-a590-6b8d5e10d546-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 1114, 'output_tokens': 32, 'total_tokens': 1146, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0,

Using Langchain Expression Language

In [35]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm
        )
rag_chain.invoke("What's the title of this paper?")

AIMessage(content='The title of the paper is "Consequences of Erudite Vernacular Utilized Irrespective of Necessity: Problems with Using Long Words Needlessly."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 1108, 'total_tokens': 1140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D5vZKNuBsw0JhMFkwhM7BXcjg1oue', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2e6b-3d50-7d62-94e8-125bd1ad37a0-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 1108, 'output_tokens': 32, 'total_tokens': 1140, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0,

Generate structured responses

In [36]:
class AnswerWithSources(BaseModel):
    """An answer to the question, with sources and reasoning."""
    answer: str = Field(description="Answer to question")
    sources: str = Field(description="Full direct text chunk from the context used to answer the question")
    reasoning: str = Field(description="Explain the reasoning of the answer based on the sources")
    
class ExtractedInfo(BaseModel):
    """Extracted information about the research article"""
    paper_title: AnswerWithSources
    paper_summary: AnswerWithSources
    publication_year: AnswerWithSources
    paper_authors: AnswerWithSources

In [37]:
rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm.with_structured_output(ExtractedInfo, strict=True)
        )

rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")

ExtractedInfo(paper_title=AnswerWithSources(answer='Consequences of Erudite Vernacular Utilized Irrespective of Necessity: Problems with Using Long Words Needlessly', sources='APPLIED COGNITIVE PSYCHOLOGY Appl. Cognit. Psychol. 20: 139–156 (2006) Published online 31 October 2005 in Wiley InterScience (www.interscience.wiley.com) DOI: 10.1002/acp.1178 Consequences of Erudite Vernacular Utilized Irrespective of Necessity: Problems with Using Long Words Needlessly DANIEL M. OPPENHEIMER* Princeton University, USA', reasoning='The title appears prominently at the beginning of the main text along with author and publication details.'), paper_summary=AnswerWithSources(answer="This research paper investigates the common writing strategy among undergraduates of using more complex vocabulary to appear intelligent. Through five experiments, the paper finds that increased linguistic complexity negatively affects readers' judgments of intelligence, regardless of writing quality or expectations. Thi

Transform response into a dataframe

In [38]:
structured_response = rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")
df = pd.DataFrame([structured_response.dict()])

# Transforming into a table with two rows: 'answer' and 'source'
answer_row = []
source_row = []
reasoning_row = []

for col in df.columns:
    answer_row.append(df[col][0]['answer'])
    source_row.append(df[col][0]['sources'])
    reasoning_row.append(df[col][0]['reasoning'])

# Create new dataframe with two rows: 'answer' and 'source'
structured_response_df = pd.DataFrame([answer_row, source_row, reasoning_row], columns=df.columns, index=['answer', 'source', 'reasoning'])
structured_response_df


/var/folders/3j/6dxlknmd5hs82yqpxtj2n3z00000gn/T/ipykernel_13160/3670145528.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  df = pd.DataFrame([structured_response.dict()])


,paper_title,paper_summary,publication_year,paper_authors
answer,Consequences of Erudite Vernacular Utilized Ir...,Most texts on writing style encourage authors ...,2006,Daniel M. Oppenheimer
source,APPLIED COGNITIVE PSYCHOLOGY Appl. Cognit. Psy...,SUMMARY Most texts on writing style encourage ...,APPLIED COGNITIVE PSYCHOLOGY Appl. Cognit. Psy...,Consequences of Erudite Vernacular Utilized Ir...
reasoning,The title is explicitly stated at the beginnin...,The summary section directly explains the rese...,The volume and year of publication are explici...,The author is clearly listed as Daniel M. Oppe...
